# Question Data Tutorial
The goal of this tutorial is to introduce some useful functions and show how to do typical tasks when working with quantitative educational data. The data included in this tutorial is reflective of what you might see if you are analyzing survey data or exam/test questions.

This tutorial assumes you already have basic knowledge of Python and Pandas and have already completed the Exam data tutorial and/or mastered the skills in that lesson. The latter part of this tutorial also assumes you have some basic statistic knowledge (hypothesis testing, p-values, statistical significance).

In this lesson, you will learn the following:
* How recode variables to new values
* Change the names of data frame columns
* Concatenate and merge data frames
* Wide vs long format for data frames
* Contigency tables
* Hypothesis testing
* Effect size

***
Created by Dr. Nicholas Young

Last modified: April 4, 2025

Python version: 3.11.9
***

As will likely be the case for most your files, we start by importing numpy, pandas, and pyplot

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Now let's read in our question files. This time we have two files, one from a morning section of a course and one from an afternoon section of a course. To minimize the amount of typing, I'll use 'm' for morning and 'a' for afternoon in the variables I store my data.

Some background on this data:
* This instructor's exam had 4 multiple choice questions and 2 free response questions. We only have the multiple choice questions.
* 1 is correct while 0 is incorrect
* Q1 and Q4 are the same on both exams but the instructor swapped the order of Q2 and Q3 on the two exams. This means that Q2 on the morning exam is Q3 on the afternoon exam and Q3 on the morning exam is Q2 on the afternoon exam.
* We also have some data about the students taking the exams. This is stored in the `class_demographics.csv` file.

In [ ]:
m_data = pd.read_csv('morning_exam.csv')
a_data = pd.read_csv('afternoon_exam.csv')

Let's start by inspecting our data and see what we are working with

In [ ]:
m_data.head()

In [ ]:
a_data.head()

You'll likely notice a problem in that Q3 is the students' responses rather than whether they are correct or not. Let's address that first.

## Changing the values of variables based on a condition.
As is often the case, our data isn't exactly the way we want it. Here, we have a column with student answers rather than whether it is correct not. We want to convert that to correct or not (1/0).

Let us assume that "B" is the correct answer to Q3 on the afternoon exam.

There are two ways to do this. Which one we want to use depends on how many different values there will be after the conversion. For a binary outcome like correct or not, we can use a logic statement. Here, we ask if Q3 is equal to B. If so, assign a 1 and otherwise, assign a zero.

In [ ]:
a_data['Q3'] =(a_data['Q3'] == "B").astype(int)

Here the `as.type(int)` is important because when we test whether each response in Q3 is B, we get a list of True and False back. True is equivalent to 1 and False is equivalent to 0 so we can convert the logic variable to an integer to get ones and zeros.

If we had multiple values were assigning to (say you were trying to categorize the alternative conception that each response targets), you could use `replace`. In `replace`, you provide a dictionary with the current values followed by the new values. This provides much more flexibility than the previous way. We also aren't restricted to a binary output.

In [ ]:
a_data['Q3'] = a_data['Q3'].replace({"A": 0, "B": 1, "C": 0, "D": 0})

If we look at the afternoon data now, we will see that all of the questions are now in correct/incorrect format

In [ ]:
a_data.head()

## Examining performance (review)

Now that the data are in the correct format, let's see on how the students did. We want to find the fraction of responses that are correct per question as well as the student's score.

Try this on your own in the cells below. If you are stuck, expand the header below called solution to see what I found or look back to the Exam data tutorial for useful functions.

In [ ]:
# use this block to put your code for the fraction of correct responses per question
# as a hint, 55% of students answered Q2 correctly in the morning class.



In [ ]:
# use this block to put your code for total score per student
# as a hint, student 101 earned a score of 2.



### Examining performance solution
Try to do this yourself before looking at the solution.

In [ ]:
m_data[['Q1', 'Q2', 'Q3', 'Q4']].mean()

In [ ]:
a_data[['Q1', 'Q2', 'Q3', 'Q4']].mean()

Because the responses are stored as correct or not, the number of correct responses is sum of the column and the number of attempts is the number of rows in the column. This is just the average so I can find the fraction of correct responses by taking the average of each column.

In [ ]:
m_data['score'] = m_data['Q1'] + m_data['Q2'] + m_data['Q3'] + m_data['Q4']
a_data['score'] = a_data['Q1'] + a_data['Q2'] + a_data['Q3'] + a_data['Q4']

In [ ]:
m_data.head()

Again, since each question is graded correct or not, each student's score is the number of questions they get correct, which is the sum of the four question columns.

## Combining data sets
So far, we've treated the two data sets as separate things. However, as the questions are the same, just in a different order, it might be useful to combine the data to get a better sense of how students are doing on these exams. For example, we might need to collect data from multiple classes in order to get a sufficient sample size.

Let's introduce some techniques that could be useful for doing this.

If I want to combine the data together into a single data frame with 200 rows instead of second data frames with 100 rows, I need each data frame to have the same column names.

We've already met this condition so we can use the `concat` function. I include `ignore_index=True` so that a new index will be created. Otherwise, the individual indices of the existing data frames will be kept and you would have two rows with each index (0-99) because `a_data` and `m_data` each have an index of 0-99.

In [ ]:
pd.concat([a_data, m_data], ignore_index=True)

Here's what that looks like without the `ignore_index=True` part. Notice that the first column ends at 99 now instead of 199. Generally it's a good idea to not have the same index twice.

In [ ]:
pd.concat([a_data, m_data])

Now that these are combined, it should be easy to do what we did before and take the column means, right? Well, not quite. If we took the column means, this would still work and not throw an error but the means wouldn't mean (pun intended) what we think they do. Because Q2 and Q3 are not the same on each exam, if we took the mean of the columns, we would answer the question of how did students do on the second and third question on each exam rather than how did student do on Q2 and Q3. (Make sure you understand the difference).

There are a few ways to proceed. One of the easiest ways is to just rename the columns so that the same questions are in columns with the same names. To prevent confusion, I'm going to use "item" to refer to the specific question and "question" to refer to the order of it on the exam.

So for the morning exam, we have
* Q1 = I1
* Q2 = I2
* Q3 = I3
* Q4 = I4

and for the afternoon exam, we have
* Q1 = I1
* Q2 = I3
* Q3 = I2
* Q4 = I4

Let's rename the columns using this scheme.

In [ ]:
a_data = a_data.rename(columns = {'Q1':'I1', 'Q2':'I2', 'Q3':'I3', 'Q4':'I4'})
m_data = m_data.rename(columns = {'Q1':'I1', 'Q3':'I2', 'Q2':'I3', 'Q4':'I4'})

First, let's see what this did to the dataframes

In [ ]:
a_data.head()

In [ ]:
m_data.head()

Python is smart enough to match columns if they are in different orders as long as the names are the same. If we `concat` now, we should get what we are hoping for.

In [ ]:
am_data = pd.concat([a_data, m_data], ignore_index=True)
am_data.head()

Now we can get the performance on each item

In [ ]:
am_data[['I1', 'I2', 'I3', 'I4']].mean()

We see that students did best on item 2 and the worst on item 4. They did about the same on items 1 and 3.

## Switching between long and wide format
There are two common formats to store data: the *wide* format and the *long* format. In the wide format, each variable has its own column while in long format, each observation has its own row, with the variable and its value as separate columns. So far, we have only worked in the wide format. Here, we will show how to convert between the two formats.

Wide format is probably how you will be working most of the time. However, long format can be useful depending on the task. If you have a lot of blank data (say there are four versions of the exam each with 10 questions and all of the versions are completely different from each other so each student only answers 10 of the 40 possible questions), long format can be a more efficient way to store data. That is, long format can also result in smaller file sizes if the long format equivlent would include a lot of missing data. This can be important if you are working with multi-GB size files or real-world data that is being added to in real-time.

Let's show how to convert between these and then an example of where long format can help.

To move from wide format to long format, we use the `melt` function. We need specify a few things to switch from wide to long. First, we need to identify the columns we want to remain as columns in the `id_vars` parameter. Next, we need to identify the columns we want to convert to a single column in the `value_vars` parameter. We specify the name of this new column in the `var_name` parameter. The values of the variables we listed in `value_vars` will be stored in a new columm whose name we specify in `value_name`.

For the `am_data`, we want to keep the `id` and `section` variables as columns so we put that into the `id_vars`. We want to collapse the items down into a single column so we put `I1`, `I2`, `I3`, and `I4` into the `value_vars` column. Because these are items, let's call the variable that will store these labels "item". Because the values are correct (1) or incorrect (0), let's store the values in a variable called "correct". The choice of "item" and "correct" as names are completely arbitary and you can call them whatever you want. Any variable not specified will in `id_vars` or `value_vars` will be dropped.

Let's see what happens when we do this.

In [ ]:
am_data2 = pd.melt(am_data, id_vars=['id','section'], value_vars=['I1', 'I2', 'I3', 'I4'],
                   var_name='item', value_name='correct')

am_data2.head(10)

Now there are four columns instead of 6. All the data are still there, just in a different format. We can confirm this by checking the shape of the data frames. We have 200 students answering four questions each. We should then have 200 x 4 = 800 rows in the long format, which we do. In this case, we didn't shrink the data frame number of cells (1200 vs 3200). That is because we didn't have any missing data and kept some columns ("id" and "section") so these are going to be repeated.

In [ ]:
print(am_data.shape)
print(am_data2.shape)

In this long format, we can still to the same type of analyses as before. For example, let's calculate the correctness per question. Here, we can group by the `item` first and then find the average of `correct`.

In [ ]:
(
am_data2
     .groupby('item')['correct']
     .mean()
)

This is the same thing we found as before! We can do basically all of the same operations in this format.

Let's try to calculate the mean, standard deviation, count, and standard error for each item

In [ ]:
am_data3 =(
            am_data2
                .groupby('item')
                .agg(
                    mean=('correct', 'mean'),
                    std =('correct', 'std'),
                    n= ('correct', 'count')
                )
                # reset the index so mean, std, and n are column names we can use later
                .reset_index()
           )
am_data3['SE'] = am_data3['std'] / np.sqrt(am_data3['n'])

am_data3.head()

This is where the format can really shine. Remember how we did the same thing in the Exam data notebook to calcuate these values for 3 exams:

```result = (
exam_data
    .groupby('group').
    agg(
    exam1_n = ('exam1', 'count'),
    exam1_mean=('exam1', 'mean'),
    exam1_std= ('exam1', 'std'),
    exam2_n = ('exam2', 'count'),
    exam2_mean=('exam2', 'mean'),
    exam2_std= ('exam2', 'std'),
    exam3_n = ('exam3', 'count'),
    exam3_mean=('exam3', 'mean'),
    exam3_std= ('exam3', 'std'),
    )
)

# calculate the standard error
result['exam1_se'] = result['exam1_std'] / np.sqrt(result['exam1_n'])
result['exam2_se'] = result['exam2_std'] / np.sqrt(result['exam2_n'])
result['exam3_se'] = result['exam3_std'] / np.sqrt(result['exam3_n'])
```
For that code, for each additional exam, we would have to add an additional four lines of code, one for the mean, standard deviation, count, and standard error.

With our data in the long format, our code to calculate these doesn't change regardless of how many questions there are. The only thing we have to change is the variables in `value_vars` in the `pd.melt` command to include more questions.

Even then, that isn't technically true. `pd.melt` assumes all columns not in `id_vars` are in `value_vars` so `pd.melt(am_data, id_vars=['id','section'], var_name='item', value_name='correct')` is equivalent to our code (assuming we didn't add other columns like the total score). That means regardless of how many questions we have, the following code will still work and give us the number of responses, the mean, the standard deviation, and standard error of every question. That is, we could calculate these statistics for 100 questions in fewer lines than it took us to calculate these statistics for three exams in the wide format.

```
am_data2 = pd.melt(am_data, id_vars=['id','section'], var_name='item', value_name='correct')

am_data3 =(
            am_data2
                .groupby('item')
                .agg(
                    mean=('correct', 'mean'),
                    std =('correct', 'std'),
                    n= ('correct', 'count')
                )
                .reset_index()
           )
am_data3['SE'] = am_data3['std'] / np.sqrt(am_data3['n'])
```

This is one particularly useful application of switching to long format. 

Given that the only variables in this long format are item and correct, we can include data from multiple exams, even on completely different topics. To keep track of where the data are coming from, we would probably want another column that records this information (for example, exam number, or course). We could then group by this variable to get the results by different exam, course, etc.

Now, if we want to switch back to to wide format, it's pretty easy to do so. We use `pd.pivot` so convert from long to wide format. We specify the columns to stay the same in the `index` parameter. The column that contains all of the new column names we want is passed to the `columns` parameter and the column that contains all the values is passed to `values`. Notice that `var_name` and `columns` are the same variables and `value_name` and `values` are the same variables for `melt` and `pivot` respectively.

I include `reset_index()` to add `id` and `section` as columns in the data frame.

In [ ]:
(
am_data2
    .pivot(index = ['id', 'section'],columns='item', values='correct')
    .reset_index()
    .head()
)

Notice this is the same as our original `am_data` data frame before we converted to wide format. I sort here so that the `id` are in the same order as the result above.

In [ ]:
am_data.sort_values('id')

### Long vs wide plotting

Let's make a plot comparing the performance on the four items.

If we are in the default wide format, here's how we could do that.

In [ ]:
# step 1: calculate the means
mean_correctness = am_data[['I1', 'I2', 'I3', 'I4']].mean()

# step 2: create the plot
plt.bar(mean_correctness.index, mean_correctness.values, color='grey') 

Now for the long format. I'm using am_data3 because we've already calculated the mean and saved it.

In [ ]:
plt.bar(am_data3['item'], am_data3['mean'])

Now, this doesn't seem too different. But if we are plotting additional things like including error bars, the long format can be useful because remember, it was easy to calculate the standard error for an arbitrary number of columns.

In [ ]:
plt.bar(am_data3['item'], am_data3['mean'])
plt.errorbar(am_data3['item'], am_data3['mean'], yerr=1.96*am_data3['SE'], capsize=5, elinewidth=1, fmt='none', color = 'black')
plt.xlabel('Item')
plt.ylabel('Correctness')
plt.title('Item correctness with 95% confidence interval')

## Merging files
Sometimes our data is stored in parts and we need to combine data sets to build the entire data set. If the columns are completedly different or not 1-to-1, concatenate probably will not work. Instead, we need a new tool.

If you are using data that is stored in a relational database (or data sets built by Dr. Young), you will probably become very familiar with this technique quickly.

This tool is `pd.merge`.

To be concrete, let's import the `class_demographics.csv` file and take a look at it.

In [ ]:
demographics = pd.read_csv('class_demographics.csv')

demographics.head()

You'll notice that this file has a column called `id` which contains three-digit id numbers, the same as the `id` column in our `am_data` data frame. That's because this `id` refers to the same student.

In [ ]:
am_data.sort_values('id').head()

That means we can link the two files to add demograhic information for each student to their responses to their exam items.

Now, you might be wondering why this information wouldn't already be included in the file. There can be a few different reasons:
* The data were collected at different times. Generally instructors don't ask their students demographics on the exam and would have collected this earlier or from some university system. Alternatively, the data might come from two different sources (e.g. consent forms and exams, exams and eLC gradebook, etc.).
* Size considerations. If there are three exams and we get the multiple choice questions from each, that would be three files. If we included demographics on each, that would mean we would have two extra copies of the demographic information. For small data sets this isn't an issue but for larger data sets, this could result in very large file sizes that are impractical to work with.
* Good data practices. If we are collecting data at multiple time points, it can be useful to separate out the data into the frequency of change. For example, demographics are unlikely to change during the semester so these are essentially fixed. Exam responses change depending on which exam the student is taking so it 'makes sense' to keep these in a separate place.

Let's actually talk about joining the two data set now. There are a few different types of ways to join two data sets. Because the pandas documentation uses SQL terminology, we will do the same. It helps to think about a venn diagram where each circle is a data set.
* **Inner join**: this is the intersection of the two data sets. If a student is in both data sets, they will remain in the data after they are inner joined. In a venn diagram, the intersection is when the circles overlap and is the *inner* part of the diagram.
* **Outer join**: this is the union of the two data sets. Any record in both data sets will be included, even if there isn't a match in the other. In a venn diagram, the union is both circles shaded and hence is the `outer` part.
* **Left join**: this is alternatively known as a left outer join. In this one, we keep all records in the first data set, regardless of whether they have a match and only keep the records in the second data frame if they have a match in the first data set.
* **Right join**: this is alternatively known as a right outer join. In this one, we keep all records in the second data set, regardless of whether they have a match and only keep the records in the first data frame if they have a match in the second data set. A right join is equivalent to a left join with the order of the data sets swapped.

Depending on the problem at hand, you will want to use different joins. Let's do an example to show the difference. To make sure we see different results, let's create a copy of the `demographics` data but remove a few of a rows as well as add a row that doesn't correspond to a student in the question data. We'll call this `demographics_drop`.

In [ ]:
demographics_drop = demographics.drop([1, 4, 7])
new_row = pd.DataFrame({'id':[500], 'group1':[0], 'group2':[0]})
demographics_drop = pd.concat([demographics_drop, new_row],ignore_index=True)

As a reminder, let's see how many rows are in the am_data currently and how many students are in this demographics data frame with a few students dropped and added.

In [ ]:
print(am_data.shape)
print(demographics_drop.shape)

Remember there are 197 students in the `am_drop` data also in the `demographics_drop` data and 3 that are not. There is also 1 student in the `demographics_drop` data not in the `am_drop` data.

First, let's do an inner join, meaning only people in both data frames will remain. We use `pd.merge` or chain to the end of our dataframe with `.merge`. We use the `how` parameter to describe what type of join (inner, out, left, right, or cross, which we ignore for now) and the `on` parameter to set which column we are matching on. The column picked here needs to be in both data sets. If the column names are different, you need to use the `left_on` and `right_on` parameters. Personnally, I recommend using the same name in both data frames to keep things easier to keep track of.

In [ ]:
am_data.merge(demographics_drop, how='inner', on='id')

Notice that now`group1` and `group2` appear in the `am_data` data frame. You'll also notice that this worked even though the two data frames were not sorted in the same way for their `id` columns. If we have concatenated, the data frames would just be placed next to each other and not matched.

We also notice there are 197 rows. This is because 197 of the 200 students were in both data frames.

Now let's do an outer join.

In [ ]:
am_data.merge(demographics_drop, how='outer', on='id')

Now we have 201 rows. This is the 197 students that are in both, the 3 only in `am_data` and the 1 only in `demographics_drop`. Notice that if there isn't a match like for student 500, `NaN` are assigned as the values for that don't have a matching value.

**You should always examine your data before and after a merge and check for NaN**. Unexpected NaN can help you determine if you did something incorrectly or your code is not behaving as expected.

Now let's do a left join. Specifically, we will left join `am_data` with `demographics_drop`.

In [ ]:
am_data.merge(demographics_drop, how='left', on='id')

Now we have 200 rows, which is the number of students in the `am_data` because we keep all of those students regardless if they are in the `demographics_drop` data. That means we will have three rows with `NaN` values for `group1` and `group2`. These are students 101, 104, and 107.

In [ ]:
am_data.merge(demographics_drop, how='left', on='id').sort_values('id').head(15)

We'll skip right joins because you can swap `am_data` and `demographics_drop` above in the left join code to achieve the same result.

We can also do the same thing with the long data. This will cause *each* instance of the student to be paired with their demographic information.

In [ ]:
am_data2.merge(demographics_drop, how='left', on='id').sort_values('id').head(16)

### Using merged files
Let's do some examples of what we can do with this data. We can calculate averages by groups as before.

First, let's assign the merged data frame to a new variable

In [ ]:
am_datam = am_data.merge(demographics_drop, how='left', on='id')

Now we let's look at the performance on each question split by `group1` and `group2`. Here's how to do that.

In [ ]:
(am_datam
     .groupby(['group1','group2'])[['I1','I2','I3', 'I4']]
     .mean()
)

From this type of analysis, we can see that students with `group1` equal to 1 and `group2` equal to 0 outperform students with `group1` equal to 1 and `group2` equal to 1 on I4.

This type of analysis is especially useful for comparing students with different demographics or other variables of interest.

## Contigency tables

Doing something similar, we can also look at performance across questions in relation to each other. For example, if a student got I1 correct, did they also get I2 correct?

Using the same `groupby` technique we have been using, it would look something like this

In [ ]:
am_datam.groupby(['I1', 'I2']).size().unstack(fill_value=0)

Here we see if that if a student got I2 correct (the last column), it is basically 50/50 whether they got I1 correct as well (70 correct, 69 incorrect). We also see that 70 students got both questions correct and 34 students got neither question correct.

This type of table is a called a contingency table or a cross tabulation. In pandas, we can make it easier with the `crosstab` function

In [ ]:
pd.crosstab(am_datam['I1'], am_datam['I2'])

If we want proportions instead, we can also do that

In [ ]:
my_contingency_table = pd.crosstab(am_datam['I1'], am_datam['I2'])

proportions = my_contingency_table / my_contingency_table.sum().sum() 

print(proportions)

## Hypothesis testing
Now that we can calculate things, let's briefly discuss how to do a statistical test so we can interpret our results.

I will assume that all assumptions and requirements of these tests are met to show you how to use the functions but before doing any test, you should look up the assumptions and requirements and perform appropriate checks. For example, does the data need to be independent, normally distributed, have equal variences, etc.?

Suppose that we want to determine whether students with `group1=1` performed better on I1 than students with `group1=0`. We can do that with a t-test.

First, let's look at the data.

In [ ]:
(am_datam
     .groupby(['group1'])[['I1']]
     .mean()
)

It clearly looks like there is a difference but it could just be due to chance. That is, if we collected the data next semester, maybe we wouldn't find the same thing. To determine that, we can get a p-value, which gives us a sense of what the chance is that this result could occur by chance if there really wasn't a difference between the two groups.

First, we need another package that contains statistical tests. Let's import that first.

In [ ]:
from scipy import stats

We will use `stat.ttest_ind` to perform the test. We use this because our data is not normally distributed (ttest) and because our data is independent (ind) because the two groups contain different people. If the same students were in both groups (say a before and after), we would use a dependent test.

To use the test, we need to create dataframes with each group separated out.

In [ ]:
group_1 = am_datam[am_datam['group1'] == 1]['I1']
group_0 = am_datam[am_datam['group1'] == 0]['I1']

The `stat.ttest_ind` function returns two values, the t statistic and the p-value. Therefore, to save the results, we need to define two variables. I'll call them t_stat and p_value.

In [ ]:
t_stat, p_value = stats.ttest_ind(group_1, group_0)

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

How do we interpret this? We can think of the t-statistic like standard errors exception we use a t-distribution rather than a normal distribution. A larger t-value means that the result is less likely to occur by chance. The p-value tells us how likely it is to obtain this t-statistic or a more extreme one by chance. Here, our test tells us we would expect to find a difference of at least 10 percentage points (55.5 vs 45.5) about 20% of the time. As our usual cutoff is $\alpha=0.05$, we would say that we do not have statistical evidence to claim that the two groups of students are performing differently on this item. So even though there was a 10 percentage point difference, it isn't statistically significant.

This is just one example of the types of tests we could do (and was technically a difference in means t test). Common ones also include difference of propotions, paired samples, and Chi-squared. For example, if we wanted to see if there is an association between how students answer I1 and I2 (the contigency table), we would use a Chi-Squared test.

Assuming we have that contigency table from earlier (`my_contingency_table`), we can do the following

In [ ]:
chi2_stat, p_value, dof, expected = stats.chi2_contingency(my_contingency_table)

print(f"Chi-squared Statistic: {chi2_stat}")
print(f"P-value: {p_value}")
print(f"Degrees of Freedom: {dof}")
print(f"Expected Frequencies:\n{expected}")

I won't go into too much detail about what all of this means (you should do that on your own if you are planning to use Chi-squared tests), but we can interpret the results the same. The p-value is not less than 0.05 so we do not have evidence of an assocation between the two items. This doesn't mean there isn't an association. We just do not have evidence of it in our data. This is the absence of evidence isn't evidence of absence argument. Be careful about reporting your results as people will get these mixed up from time to time.

## Effect size
P-values are often misused and there is a growing trend of moving away from only using p-values to determine whether than effect exists. Another common measure is the effect size. Effect size is essentially a measure of the strength of a relationship. A bigger number is generally better.

A common effect size in PER in Cohen's D, which is essentially comparing the difference in two quantites to their combined standard deviation. Any effect can then be reported as the number of standard deviations. This type of analysis is especially useful for comparing learning interventions or pre and post data.

For this example, let's suppose `group2` represents students who were taught under two different curriculums. We want to know if one curriculum improves learning compared to the other. We measure learning here based on how well perform students on a test with four items. Let's suppose `group2=0` is the standard curriculum and `group2=1` is the updated curriculum.

First, let's calculate each student's total score.

In [ ]:
am_datam['total'] = am_datam['I1'] + am_datam['I2'] + am_datam['I3'] + am_datam['I4']

Then let's compute the mean, standard deviation, and count for each group. We've done this multiple times now.

In [ ]:
group_stats =(
    am_datam
    .groupby('group2')['total']
    .agg(['mean', 'std', 'count'])
    .reset_index()
        )


Now we need to calculate Cohen's d. That is calculated as the difference in means divided by the pooled standard deviation. I won't put that formula as text and have just implemented it below. At a high level, the pooled standard deviation is how to combine the standard deviation of the two groups into one. If the two groups are the same size, then it is just the square root of the average of the two variances. Here, it isn't so simple because the sample sizes are different, and I've implemented the more general formula.

I calculate this manually because there isn't a Cohen's D function in scipy stats. There are other packages we could install though that would provide this capability.

In [ ]:
# extract means
mean1 = group_stats['mean'].iloc[0]
mean2 = group_stats['mean'].iloc[1]

# calculate Cohen's d 
cohens_d = (
    (mean2 - mean1) /  # Difference in means
    np.sqrt(((group_stats['count'].iloc[0] - 1) * group_stats['std'].iloc[0]**2 + 
              (group_stats['count'].iloc[1] - 1) * group_stats['std'].iloc[1]**2) / 
             (group_stats['count'].iloc[0] + group_stats['count'].iloc[1] - 2))  # Pooled standard deviation
)

# report the results
print("Mean of Group 1:", mean1)
print("Mean of Group 2:", mean2)
print("Cohen's d:", cohens_d)

What we care about is the d value. It is ~0.05. This means that the new curriculum increased test performance by 0.05 standard deviations. Generally 0.2 is considered small, 0.5 is medium, and 0.8 is large but it is context dependent. Either way, 0.05 is very small and suggests that this new curriculum hardly increased learning.

***
This ends the questions tutorial. You should now have a toolkit that allows you to work with a variety of types of physics education data and perform multiple types of exploratory analyses.